In [1]:
import sys
sys.path.append('../src_variable')

import road as rd
import network as nw 
import torch
import traffic_lights as tl
import junction as jn
import smoothness as sm
import generate_kvadraturen as gk
import bus
import matplotlib.pyplot as plt

In [2]:
ids1 = ["road1", "road3"]
ids2 = ["road2", "road4"]
stops1 = [("road1", 40), ("road3", 30)]
stops2 = [("road2", 45), ("road4", 35)]
times1 = [3.0, 30.0]
times2 = [5.0, 35.0]
T = 150

road1 = rd.Road(1, 50, 5, torch.tensor([50.0], requires_grad=True), [], left_pos=(-1, 0), right_pos=(0, 0),
                inflow = 0.1, id="road1")
road2 = rd.Road(1, 50, 5, torch.tensor([50.0], requires_grad=True), [], left_pos=(0, 0), right_pos=(1, 0),
                inflow = 0.1, id="road2")
road3 = rd.Road(1, 50, 5, torch.tensor([50.0], requires_grad=True), [], left_pos=(0, -1), right_pos=(0, 0),
                inflow = 0.1, id="road3")
road4 = rd.Road(1, 50, 5, torch.tensor([50.0], requires_grad=True), [], left_pos=(0, 0), right_pos=(0, 1),
                inflow = 0.1, id="road4")

trafficlight = tl.CoupledTrafficLightContinuous(False, [0], [2], [1], [3], [60.0, 40.0])
junction = jn.Junction([road1, road2, road3, road4], [0,1], [2,3], [[1.0, 0.0],[0.0, 1.0]], [], [trafficlight])
network = nw.RoadNetwork([road1, road2, road3, road4], [junction], T, optimizing = False)

bus1 = bus.Bus(ids1, stops1, times1, network)
bus2 = bus.Bus(ids2, stops2, times2, network)
bus_network = nw.RoadNetwork([road1, road2, road3, road4], [junction], T, [bus1, bus2],
                                optimizing = False, store_densities = False)
_, _, _, bus_delays = bus_network.solve_cons_law()
obj = torch.tensor(0.0)
for i in range(len(bus_delays)):
    for j in range(len(bus_delays[i])):
        obj = obj +  bus_delays[i][j]


In [3]:
obj.backward()

In [5]:
params = [road1.Vmax[0], road2.Vmax[0], road3.Vmax[0], road4.Vmax[0]] +\
        [t for t in trafficlight.cycle]

In [7]:
for p in params:
    print(p.grad)

tensor(-0.2272)
tensor(-0.2718)
tensor(-0.1216)
tensor(-0.4068)
tensor(2.2070e-25)
tensor(0.5195)
